# APIM ❤️ OpenAI

## Built-in logging and metrics

Playground to try the [buil-in logging capabilities of API Management](https://learn.microsoft.com/en-us/azure/api-management/observability). The requests are logged into Application Insights and it's easy to track request/response details and token usage with provided [notebook](openai-usage-analysis-workbook.json).

This is also to try the [emit token metric policy](https://learn.microsoft.com/en-us/azure/api-management/azure-openai-emit-token-metric-policy). The policy sends metrics to Application Insights about consumption of large language model tokens through Azure OpenAI Service APIs.

![](images/built-in-logging.gif)

Notes:
- Token count metrics include: Total Tokens, Prompt Tokens, and Completion Tokens.
- This policy supports OpenAI response streaming! Use the [streaming tool](../../tools/streaming.ipynb) to test and troubleshoot response streaming.
- Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior and troubleshoot the [policy](policy.xml).

[View policy configuration](policy.xml)

<a id='2'></a>
### 2️⃣ Create deployment using Terraform

This lab uses Terraform to declaratively define all the resources that will be deployed. Change the [variables.tf](variables.tf) directly to try different configurations.

In [ ]:
! terraform init
! terraform apply -auto-approve

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [7]:
apim_resource_gateway_url = ! terraform output -raw apim_resource_gateway_url
apim_resource_gateway_url = apim_resource_gateway_url.n
print("👉🏻 APIM Resource Gateway URL: ", apim_resource_gateway_url)

app_insights_app_id = ! terraform output -raw app_insights_app_id
app_insights_app_id = app_insights_app_id.n
print("👉🏻 Application Insights App ID: ", app_insights_app_id)

apim_subscription_key_1 = ! terraform output -raw apim_subscription_key_1
apim_subscription_key_1 = apim_subscription_key_1.n
print("👉🏻 APIM Subscription Key 1: ", apim_subscription_key_1)

apim_subscription_key_2 = ! terraform output -raw apim_subscription_key_2
apim_subscription_key_2 = apim_subscription_key_2.n
print("👉🏻 APIM Subscription Key 2: ", apim_subscription_key_2)

apim_subscription_key_3 = ! terraform output -raw apim_subscription_key_3
apim_subscription_key_3 = apim_subscription_key_3.n
print("👉🏻 APIM Subscription Key 3: ", apim_subscription_key_3)

resource_group_name = ! terraform output -raw resource_group_name
resource_group_name = resource_group_name.n
print("👉🏻 Resource Group Name: ", resource_group_name)

app_insights_resource_name = ! terraform output -raw app_insights_resource_name
app_insights_resource_name = app_insights_resource_name.n
print("👉🏻 Application Insights Resource Name: ", app_insights_resource_name)

openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"
openai_deployment_name = "gpt-4o"

👉🏻 APIM Resource Gateway URL:  https://apim-genai-330.azure-api.net
👉🏻 Application Insights App ID:  7d9e4276-c098-4d52-a23e-b022b8bfd582
👉🏻 APIM Subscription Key 1:  a41a1c78b5684165bd481a384a094179
👉🏻 APIM Subscription Key 2:  64c49e76114447f1b6b19b7da3b8f6fe
👉🏻 APIM Subscription Key 3:  c8a5704b2cb24d08a2157428c775aca5
👉🏻 Resource Group Name:  rg-apim-genai-openai-330
👉🏻 Application Insights Resource Name:  app-insights


## ⚠️ Important note about API Management Subscriptions

You will need to go to the API Management instance in the Azure portal and click each one of the three subscriptions and save them to activate them. There might be a bug preventing subscriptions to be validated.

<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [8]:
import time
import os
import json
import datetime
import requests

runs = 10
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
api_runs = []

for i in range(runs):
    print("▶️ Run:", i+1, "/", runs)
    

    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    start_time = time.time()
    response = requests.post(url, headers = {'api-key':apim_subscription_key_1}, json = messages)
    response_time = time.time() - start_time
    
    print(f"⌚ {response_time:.2f} seconds")
    # Check the response status code and apply formatting
    if 200 <= response.status_code < 300:
        status_code_str = '\x1b[1;32m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and green
    elif response.status_code >= 400:
        status_code_str = '\x1b[1;31m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and red
    else:
        status_code_str = str(response.status_code)  # No formatting

    # Print the response status with the appropriate formatting
    print("Response status:", status_code_str)
    
    print("Response headers:", response.headers)
    
    if "x-ms-region" in response.headers:
        print("x-ms-region:", '\x1b[1;31m'+response.headers.get("x-ms-region")+'\x1b[0m') # this header is useful to determine the region of the backend that served the request
        api_runs.append((response_time, response.headers.get("x-ms-region")))
    
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("Token usage:", data.get("usage"), "\n")
        print("💬 ", data.get("choices")[0].get("message").get("content"), "\n")
    else:
        print(response.text)   

▶️ Run: 1 / 10
⌚ 1.34 seconds
Response status: 200 - OK
Response headers: {'Content-Type': 'application/json', 'Date': 'Thu, 02 Jan 2025 07:10:00 GMT', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': 'f85441d8-54bb-4046-ae06-31d78bf04ab8', 'X-Content-Type-Options': 'nosniff', 'x-ratelimit-remaining-tokens': '19340', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': '3201fa75-6dfd-48cf-9d9c-0847480ac88c', 'x-ms-region': 'Sweden Central', 'azureml-model-session': 'd020-20241220211335', 'x-ratelimit-remaining-requests': '19', 'x-envoy-upstream-service-time': '631', 'x-ms-client-request-id': 'Not-Set', 'Request-Context': 'appId=cid-v1:7d9e4276-c098-4d52-a23e-b022b8bfd582'}
x-ms-region: Sweden Central
Token usage: {'completion_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_t

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [9]:
import time
from openai import AzureOpenAI

runs = 3

for i in range(runs):
    print("▶️ Run: ", i+1)

    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]

    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key_3,
        api_version=openai_api_version
    )

    start_time = time.time()

    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")
    print("💬 ", response.choices[0].message.content)


▶️ Run:  1
⌚ 0.86 seconds
💬  Oh sure, let me just consult my imaginary wristwatch... Oh wait, it’s... who cares o’clock. Look at your own dang clock, pal!
▶️ Run:  2
⌚ 0.77 seconds
💬  Oh, sure, let me just consult my non-existent watch real quick. Oh wow, it's exactly "figure-it-out-yourself" o'clock!
▶️ Run:  3
⌚ 0.98 seconds
💬  Oh sure, let me whip out my magical time-telling powers that work across space and dimensions. Oh wait, I don't actually have a clock. Why don’t *you* check?


<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [10]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = !  az monitor app-insights query --app {app_insights_app_id} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


,timestamp,apiName,apimSubscription,duration,userAgent,model,messages,completion,region,promptTokens,completionTokens,totalTokens,remainingTokens,remainingRequests
0,2025-01-02T07:10:08.7559252Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,654.12,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just consult my magical clock ...",Sweden Central,30,40,70,13400,10
1,2025-01-02T07:10:07.8027988Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,767.51,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just consult my magical time-t...",Sweden Central,30,39,69,14060,11
2,2025-01-02T07:10:06.8607529Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,728.97,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure. Just let me check my imaginary watch...",Sweden Central,30,35,65,14720,12
3,2025-01-02T07:10:05.9912405Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,646.12,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just magically know what time ...",Sweden Central,30,35,65,15380,13
4,2025-01-02T07:10:04.8574929Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,922.47,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just check my imaginary watch ...",Sweden Central,30,31,61,16040,14
5,2025-01-02T07:10:04.1075023Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,554.94,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just consult my invisible watc...",Sweden Central,30,25,55,16700,15
6,2025-01-02T07:10:03.1075001Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,785.39,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just pull out my magical unive...",Sweden Central,30,41,71,17360,16
7,2025-01-02T07:10:02.1862626Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,713.87,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just whip out my magical time-...",Sweden Central,30,34,64,18020,17
8,2025-01-02T07:10:01.0568469Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,932.58,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just use my imaginary watch th...",Sweden Central,30,61,91,18680,18
9,2025-01-02T07:09:59.7692415Z,api-azure-openai,00090137-3a5c-4ee1-aae1-b6c1cdbb6e36,1091.29,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just consult my magical crysta...",Sweden Central,30,43,73,19340,19


<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='sdk'></a>
### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [18]:
import time
from openai import AzureOpenAI
runs = 2

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_1, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 1: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_2, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 2: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_3, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 3: ", response.choices[0].message.content)


▶️ Run:  1
💬  for subscription 1:  Oh sure, let me just utilize my non-existent ability to look at clocks. Unfortunately for you, I'm just trapped in this screen with no idea what time reality is experiencing right now. Why not try something groundbreaking, like checking a clock or your phone?
💬  for subscription 2:  Oh sure, let me just consult my imaginary watch. Is that 8 o'clock in the land of make-believe? You might want to try a clock or your phone for something a bit more accurate.
💬  for subscription 3:  Oh, of course! Let me just check my non-existent watch. Actually, why don't you do something radical and look at your phone or a clock yourself? It's amazing, really.
▶️ Run:  2
💬  for subscription 1:  Oh, of course! Let me just glance at my invisible watch for you. Sorry, looks like I can't actually tell time. Maybe check a clock? They're quite good at that sort of thing!
💬  for subscription 2:  Oh sure, let me just reach through the internet and check that for you. Or, you kn

<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM.

In [19]:
import pandas as pd
import json

query = "\"" + "customMetrics  \
| where name == 'Total Tokens' \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend clientIP = tostring(parsedCustomDimensions.['Client IP']) \
| extend apiId = tostring(parsedCustomDimensions.['API ID']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend UserId = tostring(parsedCustomDimensions.['User ID']) \
| project timestamp, value, clientIP, apiId, apimSubscription, UserId \
| order by timestamp asc" + "\""

result_stdout = ! az monitor app-insights query --app {app_insights_resource_name} -g {resource_group_name} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
df = pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


,timestamp,value,clientIP,apiId,apimSubscription,UserId
0,09:07,703,176.177.25.47,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,N/A
1,09:08,67,176.177.25.47,api-azure-openai,c6a0b2b6-6e60-47f9-8622-0aaf1759de86,alex
2,09:08,151,176.177.25.47,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,alex
3,09:08,72,176.177.25.47,api-azure-openai,0328e2c9-cbcc-460e-8831-c387b509510a,alex
4,09:08,226,176.177.25.47,api-azure-openai,c6a0b2b6-6e60-47f9-8622-0aaf1759de86,N/A


## View logs and metrics on Azure Managed Grafana dashboard

You can also view the logs and metrics on the Azure Managed Grafana dashboard. Navigate to the Azure Managed Grafana. Login, then go to the dashboards section. Then import the following 2 dashboards:
1. Dashboard with ID : 16604 to vider logs and metrics from API Management.
2. Dashboard from JSON file [./grafana-dashboard-apim-openai.json](./grafana-dashboard-apim-openai.json) to view logs and metrics from Application Insights.
   Alternatively, you can also import the dashbord from the [Grafana dashboard ID: 22552](https://grafana.com/grafana/dashboards/22552).

![](images/grafana-dashboard-openai.png)